In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.duration.hazard_regression import PHReg
import pickle
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Optional, Dict, Any
import tempfile
import random

# CensorWeightCalculator Class Documentation
## Overview

The `CensorWeightCalculator` class is a tool for calculating stabilized inverse probability of censoring weights (IPCW) in survival analysis. These weights help adjust for censoring bias in longitudinal data, where subjects may drop out or be lost to follow-up before the event of interest occurs. By estimating the probability of censoring using logistic regression models, the class computes weights that can be used to produce unbiased survival estimates, such as in Kaplan-Meier estimators or Cox proportional hazards models.

The class supports flexible modeling strategies, allowing users to fit period-specific models or pool models across all periods for the numerator and/or denominator of the weights. It includes error handling, regularization, and weight stabilization features to ensure robust performance across diverse datasets.

---

## Class Definition

```python
class CensorWeightCalculator:
    def __init__(self, model_fitter, censor_event, pool_models="none"):
        self.model_fitter = model_fitter
        self.censor_event = censor_event
        self.pool_models = pool_models  # "none", "numerator", "denominator", or "both"
```

### Initialization Parameters

- **`model_fitter`**:
  - **Type**: Model class (e.g., `statsmodels.api.Logit`)
  - **Description**: The logistic regression model class used to fit censoring probability models. It must support the `.fit_regularized()` method for regularization and `.predict()` for probability estimation.
- **`censor_event`**:
  - **Type**: str
  - **Description**: The name of the column in the dataset that indicates censoring status (1 if censored, 0 otherwise).
- **`pool_models`**:
  - **Type**: str, default="none"
  - **Description**: Specifies the pooling strategy for fitting models across time periods. Options are:
    - `"none"`: Fit separate models for each period for both numerator and denominator.
    - `"numerator"`: Fit a pooled model for the numerator across all periods, with period-specific denominator models.
    - `"denominator"`: Fit a pooled model for the denominator across all periods, with period-specific numerator models.
    - `"both"`: Fit pooled models for both numerator and denominator across all periods.

### Attributes

- `self.model_fitter`: Stores the provided model fitter class.
- `self.censor_event`: Stores the censoring event column name.
- `self.pool_models`: Stores the pooling strategy.

---

## Methods

### `fit`

```python
def fit(self, data, numerator_vars, denominator_vars, id_col, period_col):
    """Calculate censoring weights."""
    # Implementation details provided in the description below
```

#### Description

The `fit` method calculates stabilized IPCW for each subject at each time period in the dataset. It fits logistic regression models (either period-specific or pooled) to predict the probability of censoring, computes weights as the ratio of probabilities of not being censored, and applies stabilization techniques to ensure robustness.

#### Parameters

- **`data`**:
  - **Type**: pandas.DataFrame
  - **Description**: The input dataset containing censoring information, covariates, and time period data.
- **`numerator_vars`**:
  - **Type**: list
  - **Description**: List of column names in `data` used as predictors in the numerator model for censoring probability.
- **`denominator_vars`**:
  - **Type**: list
  - **Description**: List of column names in `data` used as predictors in the denominator model for censoring probability. Typically includes more variables than `numerator_vars` for stabilization.
- **`id_col`**:
  - **Type**: str
  - **Description**: The name of the column in `data` that uniquely identifies subjects.
- **`period_col`**:
  - **Type**: str
  - **Description**: The name of the column in `data` that indicates the time period (e.g., follow-up time).

#### Returns

- **Type**: pandas.DataFrame
- **Description**: A DataFrame containing the calculated weights with columns:
  - `[id_col]`: Subject identifier.
  - `[period_col]`: Time period.
  - `'weight'`: The calculated censoring weight for each subject-period combination.

#### Implementation Details

1. **Data Preparation**:
   - Creates a copy of the input `data` to avoid modifying the original dataset.
2. **Pooled Model Fitting**:
   - If `pool_models` is `"numerator"` or `"both"`, fits a pooled numerator model across all periods using `_fit_pooled_model`.
   - If `pool_models` is `"denominator"` or `"both"`, fits a pooled denominator model across all periods using `_fit_pooled_model`.
3. **Period-Specific Processing**:
   - Iterates over each unique period in `period_col`:
     - Extracts data for the current period.
     - Defines a censoring indicator (`is_censored`) based on `censor_event` (1 if censored, 0 otherwise).
     - Checks for variation in `is_censored`:
       - If no variation (all 0s or all 1s), assigns weights of 1.0.
       - Otherwise, proceeds with model fitting.
4. **Model Fitting and Weight Calculation**:
   - **Numerator Model**:
     - If `pool_models` is not `"numerator"`, fits a period-specific model using `numerator_vars`.
     - Otherwise, uses the pooled numerator model.
   - **Denominator Model**:
     - If `pool_models` is not `"denominator"`, fits a period-specific model using `denominator_vars`.
     - Otherwise, uses the pooled denominator model.
   - Uses `statsmodels`' `Logit` with regularization (`alpha=0.01`) to fit models.
   - Predicts censoring probabilities (`num_probs` and `denom_probs`).
   - Clips probabilities to [0.001, 0.999] to avoid division by zero or extreme weights.
   - Calculates stabilized weights as `(1 - num_probs) / (1 - denom_probs)`.
5. **Error Handling**:
   - If model fitting fails (e.g., due to convergence issues or linear algebra errors), assigns weights of 1.0 and prints a warning.
6. **Weight Stabilization**:
   - Replaces NaN or infinite weights with 1.0.
   - Trims weights exceeding the 99th percentile to reduce variability.
7. **Output**:
   - Concatenates period-specific results into a single DataFrame and returns it.

---

### `_fit_pooled_model` (Private Method)

```python
def _fit_pooled_model(self, data, vars_list):
    """Fit a pooled model across all periods."""
    # Implementation details provided in the description below
```

#### Description

This private helper method fits a single logistic regression model across all periods to predict censoring probability. It is used for pooled models when `pool_models` is set to `"numerator"`, `"denominator"`, or `"both"`.

#### Parameters

- **`data`**:
  - **Type**: pandas.DataFrame
  - **Description**: The input dataset containing all periods.
- **`vars_list`**:
  - **Type**: list
  - **Description**: List of column names in `data` to use as predictors in the model.

#### Returns

- **Type**: Fitted model object or DummyModel
- **Description**: Returns a fitted logistic regression model or a dummy model if fitting fails or there’s no variation in the outcome.

#### Implementation Details

1. **Censoring Indicator**:
   - Creates `is_censored` based on `censor_event` (1 if censored, 0 otherwise).
2. **Variation Check**:
   - If `is_censored` has no variation (all 0s or all 1s), returns a `DummyModel` that predicts a constant probability equal to the mean of `is_censored`.
3. **Model Fitting**:
   - Adds a constant term to the predictors (`vars_list`) using `sm.add_constant`.
   - Fits a logistic regression model using `model_fitter` with regularization (`alpha=0.01`).
   - Returns the fitted model.
4. **Error Handling**:
   - If fitting fails (e.g., due to convergence issues or linear algebra errors), prints a warning and returns a `DummyModel` predicting the mean censoring probability.

#### DummyModel Definition

```python
class DummyModel:
    def predict(self, X):
        return np.ones(len(X)) * constant_prob
```

- Predicts a constant probability for all inputs, used as a fallback when model fitting is not possible.

---

## Example Usage

```python
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Sample dataset
data = pd.DataFrame({
    'id': [1, 1, 2, 2, 3, 3],
    'period': [1, 2, 1, 2, 1, 2],
    'censor': [0, 1, 0, 0, 1, 0],
    'age': [25, 26, 30, 31, 40, 41],
    'treatment': [0, 0, 1, 1, 0, 0]
})

# Initialize the calculator
calculator = CensorWeightCalculator(
    model_fitter=sm.Logit,
    censor_event='censor',
    pool_models='none'
)

# Calculate weights
weights = calculator.fit(
    data=data,
    numerator_vars=['age'],
    denominator_vars=['age', 'treatment'],
    id_col='id',
    period_col='period'
)

print(weights)
```

### Expected Output

A DataFrame like:

```
   id  period    weight
0   1       1  1.023456
1   2       1  0.987654
2   3       1  1.050000
3   1       2  1.010000
4   2       2  0.995000
5   3       2  1.030000
```

(Values are illustrative; actual weights depend on the data and model fit.)

---

## Notes

- **Purpose**: The `CensorWeightCalculator` is ideal for survival analysis tasks where censoring bias needs to be addressed, such as in medical studies or longitudinal research.
- **Stabilization**: Weights are stabilized using the ratio `(1 - num_probs) / (1 - denom_probs)`, where `numerator_vars` typically include fewer covariates than `denominator_vars` to reduce variability.
- **Robustness**: Regularization (`alpha=0.01`) prevents overfitting, while clipping probabilities and trimming weights ensures numerical stability.
- **Flexibility**: The `pool_models` parameter allows users to balance model complexity and stability based on data characteristics (e.g., sample size per period).
- **Dependencies**: Requires `pandas`, `numpy`, and `statsmodels` libraries.


In [2]:
class CensorWeightCalculator:
    def __init__(self, model_fitter, censor_event, pool_models="none"):
        self.model_fitter = model_fitter
        self.censor_event = censor_event
        self.pool_models = pool_models  # "none", "numerator", or "denominator"
    
    def fit(self, data, numerator_vars, denominator_vars, id_col, period_col):
        """Calculate censoring weights."""
        # Create a copy of the data
        data_copy = data.copy()
        
        # Create pooled models if required
        if self.pool_models in ["numerator", "both"]:
            pooled_num_model = self._fit_pooled_model(data_copy, numerator_vars)
        
        if self.pool_models in ["denominator", "both"]:
            pooled_denom_model = self._fit_pooled_model(data_copy, denominator_vars)
        
        # Get periods
        periods = sorted(data_copy[period_col].unique())
        
        # Initialize DataFrame to store weights
        weights_df = pd.DataFrame()
        
        for period in periods:
            # Get data for current period
            period_data = data_copy[data_copy[period_col] == period].copy()
            
            if len(period_data) == 0:
                continue
            
            # Define censoring indicator
            period_data['is_censored'] = (period_data[self.censor_event] == 1).astype(int)
            
            # Check if we have variation in the outcome
            if period_data['is_censored'].nunique() <= 1:
                # No variation, assign weight of 1.0
                period_data['weight'] = 1.0
            else:
                try:
                    # Fit period-specific models or use pooled models
                    if self.pool_models != "numerator":
                        # Fit numerator model for this period with regularization
                        X_num = sm.add_constant(period_data[numerator_vars])
                        num_model = sm.Logit(period_data['is_censored'], X_num).fit_regularized(
                            alpha=0.01, disp=0
                        )
                        num_probs = num_model.predict(X_num)
                    else:
                        # Use pooled numerator model
                        X_num = sm.add_constant(period_data[numerator_vars])
                        num_probs = pooled_num_model.predict(X_num)
                    
                    if self.pool_models != "denominator":
                        # Fit denominator model for this period with regularization
                        X_denom = sm.add_constant(period_data[denominator_vars])
                        denom_model = sm.Logit(period_data['is_censored'], X_denom).fit_regularized(
                            alpha=0.01, disp=0
                        )
                        denom_probs = denom_model.predict(X_denom)
                    else:
                        # Use pooled denominator model
                        X_denom = sm.add_constant(period_data[denominator_vars])
                        denom_probs = pooled_denom_model.predict(X_denom)
                    
                    # Ensure probabilities are not exactly 0 or 1
                    num_probs = np.clip(num_probs, 0.001, 0.999)
                    denom_probs = np.clip(denom_probs, 0.001, 0.999)
                    
                    # Calculate stabilized weights
                    period_data['weight'] = (1 - num_probs) / (1 - denom_probs)
                    
                except (np.linalg.LinAlgError, ValueError) as e:
                    print(f"Warning: Model fitting failed for period {period}. Setting weights to 1.0. Error: {e}")
                    period_data['weight'] = 1.0
                
            # Handle any remaining NaNs or infinities
            period_data['weight'] = period_data['weight'].fillna(1.0)
            period_data.loc[np.isinf(period_data['weight']), 'weight'] = 1.0
            
            # Trim extreme weights
            q99 = np.percentile(period_data['weight'], 99)
            period_data.loc[period_data['weight'] > q99, 'weight'] = q99
            
            # Add to weights DataFrame
            weights_df = pd.concat([weights_df, period_data[[id_col, period_col, 'weight']]])
        
        return weights_df

    def _fit_pooled_model(self, data, vars_list):
        """Fit a pooled model across all periods."""
        # Create censoring indicator
        data['is_censored'] = (data[self.censor_event] == 1).astype(int)
        
        # Check if we have variation in the outcome
        if data['is_censored'].nunique() <= 1:
            # Return a dummy model that always predicts the constant
            constant_prob = data['is_censored'].mean()
            class DummyModel:
                def predict(self, X):
                    return np.ones(len(X)) * constant_prob
            return DummyModel()
        
        try:
            # Fit model with regularization
            X = sm.add_constant(data[vars_list])
            model = sm.Logit(data['is_censored'], X).fit_regularized(alpha=0.01, disp=0)
            return model
        except (np.linalg.LinAlgError, ValueError) as e:
            print(f"Warning: Pooled model fitting failed. Creating dummy model. Error: {e}")
            # Return a dummy model
            constant_prob = data['is_censored'].mean()
            class DummyModel:
                def predict(self, X):
                    return np.ones(len(X)) * constant_prob
            return DummyModel()

# OutcomeModel Class Documentation

## Overview

The `OutcomeModel` class is designed for survival analysis using a Cox proportional hazards model. It enables users to fit a model to longitudinal data and predict survival probabilities at specified time points for new data. The class supports adjustment for additional covariates beyond the treatment variable and incorporates observation weights in the model fitting process. It leverages the `statsmodels` library's `PHReg` implementation to perform the Cox regression and provides methods to check the model's fit status and generate predictions with confidence intervals.

### Key Features
- Fits a Cox proportional hazards model to survival data.
- Predicts survival probabilities at user-specified time points with simplified confidence intervals.
- Allows adjustment for additional covariates and weighted observations.

---

## Class Definition

```python
class OutcomeModel:
    def __init__(self, adjustment_vars=None):
        self.adjustment_vars = adjustment_vars
        self.fitted_model = None
        self.model_info = None
```

### Initialization Parameters
- **`adjustment_vars`** (list of str, optional):  
  Names of additional covariates to include in the model for adjustment (e.g., `['age', 'sex']`). If `None`, only the treatment variable is used in the model.  
  Default: `None`.

---

## Methods

### `__init__(self, adjustment_vars=None)`
Initializes the `OutcomeModel` instance.

#### Parameters
- **`adjustment_vars`** (list of str, optional):  
  List of covariate names to adjust for in the model. Stored in `self.adjustment_vars`.

#### Attributes Initialized
- `self.adjustment_vars`: Stores the adjustment variables.
- `self.fitted_model`: Set to `None`, to be populated after fitting.
- `self.model_info`: Set to `None`, to store model metadata after fitting.

---

### `fit(self, data, id_col, time_col, event_col, treatment_col, weight_col)`
Fits a Cox proportional hazards model to the provided data.

#### Parameters
- **`data`** (pd.DataFrame):  
  The dataset containing survival data, with columns for identifiers, time, event status, treatment, weights, and any adjustment variables.
- **`id_col`** (str):  
  Name of the column containing unique individual identifiers (e.g., `'id'`).
- **`time_col`** (str):  
  Name of the column with time-to-event or censoring time (e.g., `'follow_up_time'`).
- **`event_col`** (str):  
  Name of the column indicating event occurrence (1 for event, 0 for censored; e.g., `'event'`).
- **`treatment_col`** (str):  
  Name of the column indicating treatment assignment (e.g., `'trial_arm'`, where 0 might represent control and 1 treatment).
- **`weight_col`** (str):  
  Name of the column with observation weights (e.g., `'weight'`, for inverse probability weighting).

#### Returns
- `self`: The fitted `OutcomeModel` instance, enabling method chaining.

#### Process
- Creates a copy of the input `data` to avoid modifying the original DataFrame.
- Constructs a formula dynamically: 
  - If `adjustment_vars` is provided, the formula becomes `"{time_col} ~ {treatment_col} + {var1} + {var2} + ..."`.
  - If `adjustment_vars` is `None`, the formula is `"{time_col} ~ {treatment_col}"`.
- Fits the Cox model using `PHReg.from_formula` from `statsmodels`, with:
  - `formula`: Specifies the model structure.
  - `data`: The copied dataset.
  - `status`: Event indicators from `data[event_col]`.
  - `weights`: Weights from `data[weight_col]`.
- Stores the fitted model in `self.fitted_model`.
- Stores additional model information in `self.model_info` as a dictionary with:
  - `'model'`: The `PHReg` model instance.
  - `'vcov'`: Variance-covariance matrix of the parameters.
  - `'formula'`: The formula used for fitting.

---

### `is_fitted(self)`
Checks whether the model has been fitted.

#### Returns
- `bool`: `True` if `self.fitted_model` is not `None`, `False` otherwise.

---

### `predict(self, data, times)`
Predicts survival probabilities for the given data at specified time points.

#### Parameters
- **`data`** (pd.DataFrame):  
  Dataset for prediction, containing the same columns as the training data (e.g., `treatment_col` and any `adjustment_vars`).
- **`times`** (list or np.array):  
  Time points at which to predict survival probabilities (e.g., `[0, 1, 2, 3, 4]`).

#### Returns
- `dict`: A dictionary containing:
  - `'times'`: The input `times` (shape: `(n_times,)`).
  - `'survival'`: 2D array of survival probabilities (shape: `(n_samples, n_times)`), where `n_samples` is the number of rows in `data`.
  - `'lower'`: Lower bounds of the simplified 95% confidence intervals (shape: `(n_samples, n_times)`).
  - `'upper'`: Upper bounds of the simplified 95% confidence intervals (shape: `(n_samples, n_times)`).

#### Raises
- `ValueError`: If the model is not fitted (`self.fitted_model` is `None`).

#### Process
- Copies the input `data` to avoid modification.
- Estimates the baseline survival function at `times` using `_estimate_baseline_survival` (shape: `(n_times,)`).
- Computes the linear predictor for `data` using `_calculate_linear_predictor` (shape: `(n_samples,)`).
- Calculates survival probabilities as `survival = baseline_surv ** exp(lp)`, using broadcasting to produce a 2D array (shape: `(n_samples, n_times)`).
- Computes simplified confidence intervals:
  - `ci_width = 1.96 * 0.1 * survival` (assuming a 95% CI with a standard error proportional to survival).
  - `lower = max(0, survival - ci_width)` to ensure non-negative probabilities.
  - `upper = min(1, survival + ci_width)` to cap probabilities at 1.

---

### `_estimate_baseline_survival(self, times)`
*Private method* to estimate the baseline survival function.

#### Parameters
- **`times`** (list or np.array):  
  Time points for estimating baseline survival.

#### Returns
- `np.array`: Baseline survival probabilities at the specified `times` (shape: `(n_times,)`).

#### Notes
- This is a simplified implementation using an exponential model with a fixed hazard rate (`lambda_hat = 0.1`).
- Formula: `S_0(t) = exp(-lambda_hat * t)`.
- In a proper implementation, this should use the baseline hazard estimated from the fitted model (e.g., `self.fitted_model.baseline_cumulative_hazard_`), but the current version uses a placeholder for simplicity.

---

### `_calculate_linear_predictor(self, data)`
*Private method* to compute the linear predictor for the input data.

#### Parameters
- **`data`** (pd.DataFrame):  
  Dataset for which to compute the linear predictor, containing columns matching the model’s exogenous variables.

#### Returns
- `np.array`: Linear predictor values for each row in `data` (shape: `(n_samples,)`).

#### Process
- Retrieves the exogenous variable names from `self.fitted_model.model.exog_names` (e.g., `['Intercept', 'trial_arm', 'x2']`).
- Constructs a design matrix `X`:
  - For `'Intercept'`, sets the column to 1.
  - For other variables, copies the corresponding column from `data`.
- Computes the linear predictor as `lp = X @ self.fitted_model.params`, where `self.fitted_model.params` are the estimated coefficients.

---

## Limitations
- **Baseline Survival**: The `_estimate_baseline_survival` method uses a simplified exponential model with a constant hazard rate (`lambda_hat = 0.1`). This does not reflect the actual baseline hazard from the data, which should be estimated from the fitted model for accuracy.
- **Confidence Intervals**: The confidence intervals in `predict` are simplified (`ci_width = 1.96 * 0.1 * survival`), assuming a standard error proportional to the survival probability. In practice, confidence intervals should account for the variance in the baseline hazard and parameter estimates (e.g., using `self.model_info['vcov']`).
- **Data Assumptions**: The `predict` method assumes `data` includes all columns specified in `treatment_col` and `adjustment_vars`. Missing columns will raise errors.
- **Proportional Hazards Assumption**: The Cox model assumes that hazard ratios are constant over time, which may not hold for all datasets.

---

## Example Usage

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sample data
data = pd.DataFrame({
    'id': range(32),
    'follow_up_time': np.random.exponential(5, 32),
    'event': np.random.binomial(1, 0.7, 32),
    'trial_arm': np.random.binomial(1, 0.5, 32),
    'x2': np.random.normal(0, 1, 32),
    'weight': np.ones(32)
})

# Initialize the model with adjustment variables
model = OutcomeModel(adjustment_vars=['x2'])

# Fit the model
model.fit(
    data=data,
    id_col='id',
    time_col='follow_up_time',
    event_col='event',
    treatment_col='trial_arm',
    weight_col='weight'
)

# Predict survival probabilities at times 0 to 10
pred_data = data.copy()
times = np.arange(11)
predictions = model.predict(pred_data, times)

# Plot average survival difference (treatment vs. control)
treatment_mask = pred_data['trial_arm'] == 1
surv_diff = (predictions['survival'][treatment_mask].mean(axis=0) - 
             predictions['survival'][~treatment_mask].mean(axis=0))
plt.plot(times, surv_diff, label='Survival Difference', color='blue')
plt.fill_between(
    times,
    predictions['lower'][treatment_mask].mean(axis=0) - predictions['upper'][~treatment_mask].mean(axis=0),
    predictions['upper'][treatment_mask].mean(axis=0) - predictions['lower'][~treatment_mask].mean(axis=0),
    color='red',
    alpha=0.2,
    label='95% CI'
)
plt.axhline(0, color='black', linestyle='--')
plt.xlabel('Follow up')
plt.ylabel('Survival Difference')
plt.title('Treatment Effect on Survival')
plt.legend()
plt.show()
```

In [3]:
class OutcomeModel:
    def __init__(self, adjustment_vars=None):
        self.adjustment_vars = adjustment_vars
        self.fitted_model = None
        self.model_info = None
    
    def fit(self, data, id_col, time_col, event_col, treatment_col, weight_col):
        """Fit a proportional hazards model."""
        model_data = data.copy()
        
        # Prepare formula
        if self.adjustment_vars:
            formula = f"{time_col} ~ {treatment_col} + " + " + ".join(self.adjustment_vars)
        else:
            formula = f"{time_col} ~ {treatment_col}"
        
        # Fit Cox PH model
        model = PHReg.from_formula(
            formula,
            data=model_data,
            status=model_data[event_col],
            weights=model_data[weight_col]
        )
        
        result = model.fit()
        
        # Store fitted model
        self.fitted_model = result
        
        # Store model info
        self.model_info = {
            'model': model,
            'vcov': result.cov_params(),
            'formula': formula
        }
        
        return self
    
    def is_fitted(self):
        """Check if model is fitted."""
        return self.fitted_model is not None
    
    def predict(self, data, times):
        """Predict survival probabilities."""
        if not self.is_fitted():
            raise ValueError("Model not fitted yet.")
        
        # Prepare prediction data
        pred_data = data.copy()
        
        # Get baseline survival (shape: (11,))
        baseline_surv = self._estimate_baseline_survival(times)
        
        # Get linear predictor (shape: (32,))
        lp = self._calculate_linear_predictor(pred_data)
        
        # Calculate survival probabilities (shape: (32, 11))
        survival = np.power(baseline_surv, np.exp(lp)[:, np.newaxis])
        
        # Calculate confidence intervals (simplified)
        ci_width = 1.96 * 0.1 * survival  # Simplified CI
        
        return {
            'times': times,
            'survival': survival,
            'lower': np.maximum(0, survival - ci_width),
            'upper': np.minimum(1, survival + ci_width)
        }
    
    def _estimate_baseline_survival(self, times):
        """Estimate baseline survival function."""
        # This is a simplified implementation
        # A proper implementation would estimate the baseline hazard from the data
        
        # For simplicity, we'll use an exponential model
        # In real implementation, this would be based on the fitted model's baseline hazard
        lambda_hat = 0.1  # Placeholder hazard rate
        return np.exp(-lambda_hat * np.array(times))
    
    def _calculate_linear_predictor(self, data):
        exog_names = self.fitted_model.model.exog_names  # ['Intercept', 'trial_arm', 'x2']
        X = pd.DataFrame(index=data.index)
        for name in exog_names:
            if name == 'Intercept':
                X[name] = 1
            else:
                X[name] = data[name]
        return X.values @ self.fitted_model.params

# `TrialSequence` Class Documentation

The `TrialSequence` class is a Python implementation designed to emulate clinical trials for causal inference, specifically for per-protocol (PP) and intention-to-treat (ITT) analyses. It mimics the functionality of the R package `TrialEmulation` by facilitating trial emulation, weight calculation, marginal structural model (MSM) fitting, and survival prediction. The class uses a `dataclass` structure to manage trial-related data and provides methods for data preparation, weight computation, trial expansion, model fitting, and prediction.

---

## Class Overview

The `TrialSequence` class is built to handle longitudinal data with repeated measures, allowing users to emulate hypothetical trials, adjust for confounding using inverse probability weights, and estimate treatment effects using survival analysis.

### Dependencies
- `pandas` for data manipulation
- `numpy` for numerical operations
- `statsmodels` for survival analysis (Cox proportional hazards model)
- `random` for random sampling
- Python `dataclasses` and `typing` for structured data management

---

## Attributes

The `TrialSequence` class uses a `dataclass` to define its attributes, providing a clean and structured way to store trial-related data.

| Attribute            | Type                  | Default | Description                                                                 |
|----------------------|-----------------------|---------|-----------------------------------------------------------------------------|
| `estimand`           | `str`                 | -       | The type of estimand to analyze: `"PP"` (per-protocol) or `"ITT"` (intention-to-treat). Required upon initialization. |
| `data`               | `Optional[pd.DataFrame]` | `None`  | The input dataset containing longitudinal trial data.               |
| `id_col`             | `Optional[str]`       | `None`  | Name of the column in `data` identifying unique individuals.        |
| `period_col`         | `Optional[str]`       | `None`  | Name of the column in `data` indicating time periods.               |
| `treatment_col`      | `Optional[str]`       | `None`  | Name of the column in `data` indicating treatment assignment (binary: 0 or 1). |
| `outcome_col`        | `Optional[str]`       | `None`  | Name of the column in `data` indicating the outcome (event indicator: 0 or 1). |
| `eligible_col`       | `Optional[str]`       | `None`  | Name of the column in `data` indicating eligibility for trial emulation (0 or 1). |
| `switch_weights`     | `Optional[pd.DataFrame]` | `None`  | DataFrame containing switch weights to adjust for treatment switching (PP analysis). |
| `censor_weights`     | `Optional[pd.DataFrame]` | `None`  | DataFrame containing censoring weights to adjust for informative censoring. |
| `combined_weights`   | `Optional[pd.DataFrame]` | `None`  | DataFrame containing combined weights (product of switch and censor weights). |
| `outcome_model`      | `Optional[Any]`       | `None`  | The fitted outcome model object (e.g., an instance of `OutcomeModel`) for survival analysis. |
| `expansion`          | `Optional[pd.DataFrame]` | `None`  | Expanded trial data created by `expand_trials`, used for fitting the MSM. |
| `expansion_options`  | `Optional[Dict]`      | `None`  | Dictionary containing options for trial expansion, such as chunk size and output handler. |

---

## Methods

### `__init__(estimand: str)`
Initializes a new `TrialSequence` instance.

#### Parameters
- `estimand` (`str`): The type of estimand to analyze. Must be `"PP"` (per-protocol) or `"ITT"` (intention-to-treat).

#### Example
```python
trial = TrialSequence(estimand="ITT")
```

---

### `set_data(data: pd.DataFrame, id: str, period: str, treatment: str, outcome: str, eligible: str) -> TrialSequence`
Sets the input data and column names for the trial sequence.

#### Parameters
- `data` (`pd.DataFrame`): The input dataset containing longitudinal trial data.
- `id` (`str`): Name of the column in `data` identifying unique individuals.
- `period` (`str`): Name of the column in `data` indicating time periods.
- `treatment` (`str`): Name of the column in `data` indicating treatment assignment (binary: 0 or 1).
- `outcome` (`str`): Name of the column in `data` indicating the outcome (event indicator: 0 or 1).
- `eligible` (`str`): Name of the column in `data` indicating eligibility for trial emulation (0 or 1).

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Example
```python
data = pd.DataFrame({
    'id': [1, 1, 2, 2],
    'period': [0, 1, 0, 1],
    'treatment': [0, 1, 0, 0],
    'outcome': [0, 1, 0, 0],
    'eligible': [1, 1, 1, 1]
})
trial.set_data(data, id="id", period="period", treatment="treatment", outcome="outcome", eligible="eligible")
```

---

### `set_switch_weight_model(numerator: str, denominator: str, model_fitter: Any) -> TrialSequence`
Sets up and calculates switch weights to adjust for treatment switching (used in PP analysis).

#### Parameters
- `numerator` (`str`): R-style formula string for the numerator model (e.g., `"~ age"`).
- `denominator` (`str`): R-style formula string for the denominator model (e.g., `"~ age + x1"`).
- `model_fitter` (`Any`): A model fitter object (e.g., `StatsGlmLogit`) to fit logistic regression models for switch weights.

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Notes
- Switch weights are calculated using stabilized inverse probability weights to adjust for treatment switching over time.
- The `model_fitter` should have a `fit` method that accepts the data, treatment column, numerator and denominator variables, and ID and period columns.

#### Example
```python
trial.set_switch_weight_model(
    numerator="~ age",
    denominator="~ age + x1",
    model_fitter=StatsGlmLogit(save_path="switch_models")
)
```

---

### `set_censor_weight_model(censor_event: str, numerator: str, denominator: str, pool_models: str, model_fitter: Any) -> TrialSequence`
Sets up and calculates censoring weights to adjust for informative censoring.

#### Parameters
- `censor_event` (`str`): Name of the censoring indicator column in `data` (0 or 1).
- `numerator` (`str`): R-style formula string for the numerator model (e.g., `"~ x2"`).
- `denominator` (`str`): R-style formula string for the denominator model (e.g., `"~ x2 + x1"`).
- `pool_models` (`str`): Strategy for pooling models across periods: `"none"`, `"numerator"`, or `"denominator"`.
- `model_fitter` (`Any`): A model fitter object (e.g., `StatsGlmLogit`) to fit logistic regression models for censoring weights.

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Notes
- Censoring weights are calculated using stabilized inverse probability of censoring weights.
- Requires a `CensorWeightCalculator` object to handle the weight computation.

#### Example
```python
trial.set_censor_weight_model(
    censor_event="censored",
    numerator="~ x2",
    denominator="~ x2 + x1",
    pool_models="none",
    model_fitter=StatsGlmLogit(save_path="censor_models")
)
```

---

### `calculate_weights() -> TrialSequence`
Combines switch and censor weights into a single set of weights for analysis.

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Raises
- `ValueError`: If neither `switch_weights` nor `censor_weights` have been calculated.

#### Notes
- Creates a DataFrame of all possible individual-period combinations.
- Merges switch and censor weights, filling missing weights with 1.0.
- Computes combined weights as the product of switch and censor weights.

#### Example
```python
trial.calculate_weights()
```

---

### `set_outcome_model(adjustment_terms: Optional[str] = None) -> TrialSequence`
Sets up the outcome model for survival analysis.

#### Parameters
- `adjustment_terms` (`Optional[str]`): R-style formula string for adjustment terms (e.g., `"~ x2"`). If `None`, no adjustment terms are used.

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Notes
- Creates an `OutcomeModel` instance, optionally with adjustment variables.

#### Example
```python
trial.set_outcome_model(adjustment_terms="~ x2")
```

---

### `set_expansion_options(output: Optional[Callable] = None, chunk_size: int = 500) -> TrialSequence`
Sets options for trial data expansion.

#### Parameters
- `output` (`Optional[Callable]`): An output handler function to process expanded data (default: `None`).
- `chunk_size` (`int`): Number of individuals to process per chunk during expansion (default: 500).

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Example
```python
trial.set_expansion_options(output=save_to_datatable(), chunk_size=500)
```

---

### `expand_trials() -> TrialSequence`
Expands the trial data for analysis, creating a dataset of emulated trials.

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Raises
- `ValueError`: If `expansion_options` are not set (call `set_expansion_options` first).

#### Notes
- Processes individuals in chunks to manage memory usage.
- Calls `_expand_individuals` to generate trial records for each chunk.

#### Example
```python
trial.expand_trials()
```

---

### `_expand_individuals(data: pd.DataFrame) -> pd.DataFrame`
Creates expanded trial data for a subset of individuals.

#### Parameters
- `data` (`pd.DataFrame`): Subset of the input data for the current chunk of individuals.

#### Returns
- `pd.DataFrame`: Expanded trial data for the given individuals.

#### Notes
- For each individual, identifies eligible periods and creates trial records.
- Computes survival time and event status for each trial.
- Includes baseline covariates and weights from `combined_weights`.

---

### `load_expanded_data(seed: Optional[int] = None, p_control: float = 0.5) -> TrialSequence`
Loads the expanded trial data and applies sampling weights.

#### Parameters
- `seed` (`Optional[int]`): Random seed for reproducibility (default: `None`).
- `p_control` (`float`): Probability of assignment to the control arm (default: 0.5).

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Raises
- `ValueError`: If `expansion` is `None` (call `expand_trials` first).

#### Notes
- Adds sampling weights to balance treatment assignment probabilities.

#### Example
```python
trial.load_expanded_data(seed=1234, p_control=0.5)
```

---

### `fit_msm(weight_cols: List[str], modify_weights: Optional[Callable] = None) -> TrialSequence`
Fits a marginal structural model (MSM) using a Cox proportional hazards model.

#### Parameters
- `weight_cols` (`List[str]`): List of column names for additional weights (e.g., `["sample_weight"]`).
- `modify_weights` (`Optional[Callable]`): Function to modify combined weights (e.g., winsorization at the 99th percentile).

#### Returns
- `self`: Returns the `TrialSequence` instance for method chaining.

#### Raises
- `ValueError`: If `expansion` is `None` (call `expand_trials` first).

#### Notes
- Fits the model using `survival_time` and `event` columns from the expanded data.
- Combines weights from `combined_weights` and additional `weight_cols`.

#### Example
```python
trial.fit_msm(
    weight_cols=["sample_weight"],
    modify_weights=lambda w: np.minimum(w, np.quantile(w, 0.99))
)
```

---

### `predict(newdata: pd.DataFrame, predict_times: List[int], type: str = "survival") -> Dict`
Predicts survival outcomes based on the fitted MSM.

#### Parameters
- `newdata` (`pd.DataFrame`): Data for prediction, with the same structure as the training data.
- `predict_times` (`List[int]`): List of time points at which to predict survival probabilities (e.g., `[0, 1, ..., 10]`).
- `type` (`str`): Type of prediction (default: `"survival"`).

#### Returns
- `Dict`: A dictionary containing:
  - `arm_0`: Survival predictions for the control arm (treatment = 0).
  - `arm_1`: Survival predictions for the treatment arm (treatment = 1).
  - `difference`: DataFrame with columns `followup_time`, `survival_diff`, `2.5%`, and `97.5%`, representing the difference in survival probabilities and confidence intervals.

#### Raises
- `ValueError`: If the outcome model is not fitted (call `fit_msm` first).

#### Example
```python
prediction_data = trial.expansion[trial.expansion['trial_period'] == 1]
preds = trial.predict(
    newdata=prediction_data,
    predict_times=list(range(11)),
    type="survival"
)
```

---

### `_formula_to_vars(formula: Union[str, Any]) -> List[str]`
Converts an R-style formula string to a list of variable names.

#### Parameters
- `formula` (`Union[str, Any]`): R-style formula string (e.g., `"~ age + x1"`).

#### Returns
- `List[str]`: List of variable names extracted from the formula (e.g., `["age", "x1"]`).

#### Notes
- Handles formulas by removing the `~` and splitting on `+`.

---

## Example Usage
Below is a complete example of using the `TrialSequence` class to emulate trials, fit an MSM, and predict survival differences:

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.duration.hazard_regression import PHReg

# Create a sample dataset
data = pd.DataFrame({
    'id': [1, 1, 2, 2],
    'period': [0, 1, 0, 1],
    'treatment': [0, 1, 0, 0],
    'outcome': [0, 1, 0, 0],
    'eligible': [1, 1, 1, 1],
    'censored': [0, 0, 0, 1],
    'age': [30, 30, 40, 40],
    'x1': [0.5, 0.6, 0.3, 0.4]
})

# Initialize TrialSequence for ITT analysis
trial_itt = TrialSequence(estimand="ITT")

# Set data
trial_itt.set_data(
    data=data,
    id="id",
    period="period",
    treatment="treatment",
    outcome="outcome",
    eligible="eligible"
)

# Set censoring weights
trial_itt.set_censor_weight_model(
    censor_event="censored",
    numerator="~ age",
    denominator="~ age + x1",
    pool_models="none",
    model_fitter=StatsGlmLogit()
)

# Calculate weights
trial_itt.calculate_weights()

# Set outcome model with adjustment
trial_itt.set_outcome_model(adjustment_terms="~ age")

# Set expansion options and expand trials
trial_itt.set_expansion_options(chunk_size=500)
trial_itt.expand_trials()

# Load expanded data
trial_itt.load_expanded_data(seed=1234, p_control=0.5)

# Fit MSM
trial_itt.fit_msm(weight_cols=["sample_weight"])

# Predict survival differences
prediction_data = trial_itt.expansion[trial_itt.expansion['trial_period'] == 1].copy()
prediction_data['Intercept'] = 1  # Ensure intercept for prediction
preds = trial_itt.predict(
    newdata=prediction_data,
    predict_times=list(range(11)),
    type="survival"
)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(preds['difference']['followup_time'], preds['difference']['survival_diff'], label="Survival Difference")
plt.plot(preds['difference']['followup_time'], preds['difference']['2.5%'], 'r--', label="95% CI")
plt.plot(preds['difference']['followup_time'], preds['difference']['97.5%'], 'r--')
plt.axhline(0, color='blue', linestyle='--')
plt.xlabel("Follow up")
plt.ylabel("Survival difference")
plt.title("Treatment Effect on Survival")
plt.legend()
plt.grid(True)
plt.show()
```

---

## Notes
- **Weight Calculations**: The class supports inverse probability weights for treatment switching (PP) and censoring (PP and ITT), crucial for unbiased causal inference.
- **Trial Expansion**: The `expand_trials` method emulates randomized trials by creating a dataset where each eligible period for an individual starts a new trial.
- **Survival Analysis**: Uses `statsmodels`’ `PHReg` for fitting a Cox proportional hazards model, allowing for time-to-event analysis.
- **Flexibility**: The class is designed to handle large datasets by processing individuals in chunks during trial expansion.

---

## Limitations
- The current implementation assumes a binary treatment variable (`0` for control, `1` for treatment).
- Confidence intervals in `predict` are simplified and may not be statistically rigorous (consider using bootstrap methods for better intervals).
- The baseline survival estimation in `OutcomeModel` may need enhancement to align with R’s event-driven approach (e.g., using Kaplan-Meier or Breslow estimator).

In [4]:
# Dataclass to mimic the trial_sequence object in R
@dataclass
class TrialSequence:
    estimand: str  # "PP" or "ITT"
    data: Optional[pd.DataFrame] = None
    id_col: Optional[str] = None
    period_col: Optional[str] = None
    treatment_col: Optional[str] = None
    outcome_col: Optional[str] = None
    eligible_col: Optional[str] = None
    switch_weights: Optional[pd.DataFrame] = None
    censor_weights: Optional[pd.DataFrame] = None
    combined_weights: Optional[pd.DataFrame] = None
    outcome_model: Optional[Any] = None
    expansion: Optional[pd.DataFrame] = None
    expansion_options: Optional[Dict] = None
    
    def set_data(self, data, id, period, treatment, outcome, eligible):
        """Set the data and column names for the trial sequence."""
        self.data = data
        self.id_col = id
        self.period_col = period
        self.treatment_col = treatment
        self.outcome_col = outcome
        self.eligible_col = eligible
        return self
    
    def set_switch_weight_model(self, numerator, denominator, model_fitter):
        """Set the switch weight model specifications."""
        # Convert R formula strings to lists of variable names
        num_vars = self._formula_to_vars(numerator)
        denom_vars = self._formula_to_vars(denominator)
        
        # Calculate switch weights
        self.switch_weights = model_fitter.fit(
            self.data, 
            self.treatment_col,
            num_vars,
            denom_vars,
            self.id_col,
            self.period_col
        )
        
        return self
    
    def set_censor_weight_model(self, censor_event, numerator, denominator, pool_models, model_fitter):
        """Set the censoring weight model specifications."""
        # Convert R formula strings to lists of variable names
        num_vars = self._formula_to_vars(numerator)
        denom_vars = self._formula_to_vars(denominator)
        
        # Calculate censoring weights
        censor_calculator = CensorWeightCalculator(
            model_fitter=model_fitter,
            censor_event=censor_event,
            pool_models=pool_models
        )
        
        self.censor_weights = censor_calculator.fit(
            self.data,
            num_vars,
            denom_vars,
            self.id_col,
            self.period_col
        )
        
        return self
    
    def calculate_weights(self):
        """Combine switch and censor weights."""
        if self.switch_weights is None and self.censor_weights is None:
            raise ValueError("No weights have been calculated yet.")
        
        # Create a base DataFrame with all patient-periods
        all_periods = pd.DataFrame({
            self.id_col: self.data[self.id_col].unique()
        }).merge(
            pd.DataFrame({self.period_col: self.data[self.period_col].unique()}),
            how='cross'
        )
        
        # Merge with switch weights if available
        if self.switch_weights is not None:
            all_periods = pd.merge(
                all_periods,
                self.switch_weights[[self.id_col, self.period_col, 'weight']],
                on=[self.id_col, self.period_col],
                how='left'
            )
            all_periods.rename(columns={'weight': 'switch_weight'}, inplace=True)
            all_periods['switch_weight'].fillna(1.0, inplace=True)
        else:
            all_periods['switch_weight'] = 1.0
        
        # Merge with censor weights if available
        if self.censor_weights is not None:
            all_periods = pd.merge(
                all_periods,
                self.censor_weights[[self.id_col, self.period_col, 'weight']],
                on=[self.id_col, self.period_col],
                how='left'
            )
            all_periods.rename(columns={'weight': 'censor_weight'}, inplace=True)
            all_periods['censor_weight'].fillna(1.0, inplace=True)
        else:
            all_periods['censor_weight'] = 1.0
        
        # Calculate combined weight
        all_periods['weight'] = all_periods['switch_weight'] * all_periods['censor_weight']
        
        # Store combined weights
        self.combined_weights = all_periods
        
        return self
    
    def set_outcome_model(self, adjustment_terms=None):
        """Set up the outcome model for survival analysis."""
        if adjustment_terms is None:
            self.outcome_model = OutcomeModel()
        else:
            adj_vars = self._formula_to_vars(adjustment_terms)
            self.outcome_model = OutcomeModel(adjustment_vars=adj_vars)
        
        return self
    
    def set_expansion_options(self, output=None, chunk_size=500):
        """Set options for trial expansion."""
        self.expansion_options = {
            'output_handler': output,
            'chunk_size': chunk_size
        }
        return self
    
    def expand_trials(self):
        """Expand the trial data for analysis."""
        if self.expansion_options is None:
            raise ValueError("Expansion options not set. Call set_expansion_options first.")
        
        # Get unique individuals
        individuals = self.data[self.id_col].unique()
        
        # Process in chunks
        chunk_size = self.expansion_options['chunk_size']
        results = []
        
        for i in range(0, len(individuals), chunk_size):
            chunk_ids = individuals[i:i+chunk_size]
            
            # Filter data for current chunk
            chunk_data = self.data[self.data[self.id_col].isin(chunk_ids)].copy()
            
            # Create expanded data
            expanded = self._expand_individuals(chunk_data)
            results.append(expanded)
        
        # Combine results
        self.expansion = pd.concat(results, ignore_index=True)
        
        return self
    
    def _expand_individuals(self, data):
        """Create expanded data for a set of individuals with survival times."""
        expanded_data = []
        
        for id_val in data[self.id_col].unique():
            # Get individual's data, sorted by period
            indiv_data = data[data[self.id_col] == id_val].sort_values(by=self.period_col)
            
            # Find event time: first period where outcome == 1 or censored == 1
            event_rows = indiv_data[(indiv_data[self.outcome_col] == 1) | (indiv_data['censored'] == 1)]
            if not event_rows.empty:
                event_time = event_rows.iloc[0][self.period_col]
                event_status = event_rows.iloc[0][self.outcome_col]
            else:
                # No event or censoring; assume censored at last period
                event_time = indiv_data[self.period_col].max()
                event_status = 0
            
            # Eligible start periods are before or at event_time
            eligible_data = indiv_data[(indiv_data[self.eligible_col] == 1) & 
                                    (indiv_data[self.period_col] <= event_time)]
            
            for _, row in eligible_data.iterrows():
                start_period = row[self.period_col]
                trial_arm = row[self.treatment_col]
                survival_time = event_time - start_period
                event = event_status
                
                # Create trial record
                record = {
                    self.id_col: id_val,
                    'trial_period': start_period,
                    'trial_arm': trial_arm,
                    'survival_time': survival_time,
                    'event': event,
                }
                
                # Include baseline covariates from start period
                for col in self.data.columns:
                    if col not in [self.id_col, self.period_col, self.treatment_col, 
                                self.outcome_col, 'censored', self.eligible_col]:
                        record[col] = row[col]
                
                # Include weight for this trial from combined_weights
                if self.combined_weights is not None:
                    weight_row = self.combined_weights[
                        (self.combined_weights[self.id_col] == id_val) & 
                        (self.combined_weights[self.period_col] == start_period)
                    ]
                    record['weight'] = weight_row['weight'].values[0] if not weight_row.empty else 1.0
                else:
                    record['weight'] = 1.0
                
                expanded_data.append(record)
        
        return pd.DataFrame(expanded_data)
    
    def load_expanded_data(self, seed=None, p_control=0.5):
        """Load expanded data and apply sampling weights."""
        if self.expansion is None:
            raise ValueError("No expanded data available. Call expand_trials first.")
        
        # Set random seed if provided
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # Sample from expanded data
        expanded_data = self.expansion.copy()
        
        # Calculate sampling weights based on treatment assignment
        expanded_data['sample_weight'] = np.where(
            expanded_data['trial_arm'] == 0,  # Assuming 0 is control
            1.0 / p_control,
            1.0 / (1.0 - p_control)
        )
        
        # Store back to expansion
        self.expansion = expanded_data
        
        return self
    
    def fit_msm(self, weight_cols, modify_weights=None):
        """
        Fit a marginal structural model using a Cox proportional hazards model.

        Parameters:
        - weight_cols: List of column names for additional weights (e.g., censoring weights).
        - modify_weights: Optional function to modify combined weights (e.g., winsorization).
        """
        # Check if expanded data exists
        if self.expansion is None:
            raise ValueError("No expanded data available. Call expand_trials first.")
        
        # Prepare a copy of the expanded data
        model_data = self.expansion.copy()
        
        # Combine weights: start with trial weights, then multiply by additional weights
        model_data['combined_weight'] = model_data['weight']
        for col in weight_cols:
            if col in model_data.columns:
                model_data['combined_weight'] *= model_data[col]
        
        # Apply weight modification if provided
        if modify_weights is not None:
            model_data['combined_weight'] = modify_weights(model_data['combined_weight'])
        
        # Ensure the outcome model is set
        if self.outcome_model is None:
            self.set_outcome_model()
        
        # Define the time variable (endog), event status (status), and covariates (exog)
        endog = model_data['survival_time']  # Time-to-event variable
        status = model_data['event']         # Event indicator (1 if event occurred, 0 if censored)
        exog = sm.add_constant(model_data[['trial_arm']])  # Covariates: intercept + treatment
        
        # Include adjustment variables if specified
        if self.outcome_model.adjustment_vars:
            exog = pd.concat([exog, model_data[self.outcome_model.adjustment_vars]], axis=1)
        
        # Fit the Cox PH model
        model = PHReg(endog, exog, status=status, weights=model_data['combined_weight'])
        self.outcome_model.fitted_model = model.fit()
        
        # Store model information
        self.outcome_model.model_info = {
            'model': model,
            'vcov': self.outcome_model.fitted_model.cov_params(),
            'exog_names': exog.columns.tolist()
        }
        
        return self
    
    def predict(self, newdata, predict_times, type="survival"):
        """Predict outcomes based on the fitted model."""
        if self.outcome_model is None or not self.outcome_model.is_fitted():
            raise ValueError("Outcome model not fitted. Call fit_msm first.")
        
        # Prepare prediction data
        pred_data = newdata.copy()
        
        # Make predictions for each treatment arm
        results = {}
        for arm in [0, 1]:  # Binary treatment (0=control, 1=treatment)
            pred_data['trial_arm'] = arm
            surv_curves = self.outcome_model.predict(pred_data, predict_times)
            # Compute mean over individuals (axis=0) for each time point
            mean_survival = np.mean(surv_curves['survival'], axis=0)
            mean_lower = np.mean(surv_curves['lower'], axis=0)
            mean_upper = np.mean(surv_curves['upper'], axis=0)
            results[f'arm_{arm}'] = {
                'times': surv_curves['times'],
                'survival': mean_survival,
                'lower': mean_lower,
                'upper': mean_upper
            }
        
        # Calculate difference (treatment effect)
        diff_data = pd.DataFrame({
            'followup_time': predict_times,
            'survival_diff': results['arm_1']['survival'] - results['arm_0']['survival'],
            '2.5%': results['arm_1']['lower'] - results['arm_0']['upper'],  # Simplified CI
            '97.5%': results['arm_1']['upper'] - results['arm_0']['lower']  # Simplified CI
        })
        
        return {
            'arm_0': results['arm_0'],
            'arm_1': results['arm_1'],
            'difference': diff_data
        }
    
    def _formula_to_vars(self, formula):
        """Convert an R-style formula to a list of variable names."""
        if isinstance(formula, str):
            # Remove ~ and split by +
            parts = formula.replace("~", "").split("+")
            return [part.strip() for part in parts]
        else:
            # For our example, we'll handle the simpler case where the formula is a string like "~ age + x1 + x3"
            # For the case where formula is passed as a tilde object in R, we just extract the string after the tilde
            formula_str = formula.replace("~", "")
            parts = formula_str.split("+")
            return [part.strip() for part in parts]

# `StatsGlmLogit` Class Documentation

## Overview

The `StatsGlmLogit` class is a Python utility designed to fit logistic regression models and calculate stabilized inverse probability weights (IPW) for longitudinal data analysis. These weights are commonly used in causal inference to adjust for confounding due to treatment switching over time. The class is particularly suited for survival analysis contexts, such as estimating treatment effects on survival outcomes, and supports optional saving of fitted models for later use or inspection.

### Key Features
- Fits logistic regression models for numerator (simpler) and denominator (full) components to calculate stabilized weights.
- Handles longitudinal data with multiple time periods and treatment switching.
- Optionally saves fitted models to a specified directory.
- Returns a DataFrame with stabilized weights for each individual and period.

---

## Class Definition

```python
class StatsGlmLogit:
    def __init__(self, save_path=None):
        self.save_path = save_path
        if save_path and not os.path.exists(save_path):
            os.makedirs(save_path)
```

### Initialization Parameters
- **`save_path`** (str, optional):  
  Path to a directory where fitted logistic regression models will be saved. If provided, the directory is created if it does not exist.  
  **Default**: `None` (models are not saved).

---

## Methods

### `__init__(self, save_path=None)`
Initializes the `StatsGlmLogit` instance.

#### Parameters
- **`save_path`** (str, optional):  
  Directory path for saving fitted models. If specified and the directory does not exist, it is created using `os.makedirs`.

#### Attributes Initialized
- **`self.save_path`**: Stores the provided `save_path` or `None` if not specified.

---

### `fit(self, data, treatment_col, numerator_vars, denominator_vars, id_col, period_col)`
Fits logistic regression models to calculate stabilized weights for each time period based on treatment switching.

#### Parameters
- **`data`** (pd.DataFrame):  
  Input longitudinal dataset containing columns for treatment, covariates, individual identifiers, and time periods.
- **`treatment_col`** (str):  
  Name of the column indicating treatment assignment (e.g., `'treatment'`), typically binary (0 or 1).
- **`numerator_vars`** (list of str):  
  List of covariate names for the numerator logistic regression model (e.g., `['age', 'baseline_severity']`). This model is simpler, often including time-invariant or baseline covariates.
- **`denominator_vars`** (list of str):  
  List of covariate names for the denominator logistic regression model (e.g., `['age', 'baseline_severity', 'time_varying_covariate']`). This model is more comprehensive, including covariates influencing treatment switching.
- **`id_col`** (str):  
  Name of the column with unique individual identifiers (e.g., `'id'`).
- **`period_col`** (str):  
  Name of the column indicating time periods (e.g., `'period'`), with integer values representing discrete time points.

#### Returns
- **`pd.DataFrame`**: A DataFrame containing stabilized weights with the following columns:
  - `[id_col]`: Individual identifier.
  - `[period_col]`: Time period.
  - `'weight'`: Stabilized weight for each individual-period combination.

#### Process
1. **Extract Unique Periods**:  
   Retrieves and sorts unique time periods from `data[period_col]`.
2. **Initialize Weights DataFrame**:  
   Creates an empty DataFrame to store weights across all periods.
3. **Iterate Over Periods**:  
   Loops through periods starting from the second one (since switching requires a prior period):
   - **Filter Current Period Data**: Extracts data for the current period; skips if empty.
   - **Identify Previous Period**: Determines the previous period from the sorted list.
   - **Merge with Previous Data**: Combines current period data with previous period’s treatment data using `id_col`.
   - **Detect Treatment Switching**: Creates a `'switched'` column (1 if treatment differs from the previous period, 0 otherwise).
   - **Fit Numerator Model**: Fits a logistic regression model using `numerator_vars` to predict switching probability, suppressing output (`disp=0`).
   - **Save Numerator Model**: If `save_path` is specified, saves the model as a pickle file (e.g., `num_model_period_1.pkl`).
   - **Fit Denominator Model**: Fits a logistic regression model using `denominator_vars` to predict switching probability, suppressing output.
   - **Save Denominator Model**: If `save_path` is specified, saves the model as a pickle file (e.g., `denom_model_period_1.pkl`).
   - **Calculate Probabilities**: Computes predicted probabilities of switching from both models.
   - **Compute Stabilized Weights**: Calculates weights as the ratio `num_probs / denom_probs`, replacing NaN values (e.g., from division by zero) with 1.0.
   - **Append Weights**: Adds the current period’s weights to the weights DataFrame.
4. **Return Weights**: Concatenates and returns the weights DataFrame for all periods.

#### Dependencies
- **`pandas`**: For data manipulation (`pd.DataFrame`, `pd.merge`, `pd.concat`).
- **`statsmodels.api`**: For logistic regression (`sm.Logit`, `sm.add_constant`).
- **`os`**: For directory handling (`os.path.exists`, `os.makedirs`).
- **`pickle`**: For saving models (`pickle.dump`).

---

## Example Usage

```python
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import pickle

# Sample longitudinal data
data = pd.DataFrame({
    'id': [1, 1, 2, 2, 3, 3],
    'period': [0, 1, 0, 1, 0, 1],
    'treatment': [0, 1, 0, 0, 1, 1],
    'age': [25, 25, 30, 30, 35, 35],
    'baseline_severity': [0.5, 0.5, 0.6, 0.6, 0.7, 0.7],
    'time_varying_covariate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
})

# Initialize StatsGlmLogit with a save path
model_fitter = StatsGlmLogit(save_path='fitted_models')

# Fit models and calculate weights
weights = model_fitter.fit(
    data=data,
    treatment_col='treatment',
    numerator_vars=['age', 'baseline_severity'],
    denominator_vars=['age', 'baseline_severity', 'time_varying_covariate'],
    id_col='id',
    period_col='period'
)

print(weights)
```

### Expected Output
A DataFrame such as:

```
   id  period    weight
0   1       1  1.023456
1   2       1  0.987654
2   3       1  1.050000
```

*Note*: Actual weight values depend on the data and model fit; the above are illustrative.

---

## Notes

- **Purpose**: The class calculates stabilized weights to adjust for confounding in longitudinal studies with time-varying treatments, often for causal inference in survival analysis.
- **Stabilized Weights**: Computed as the ratio of probabilities from a simpler numerator model and a more comprehensive denominator model, these weights balance treatment switching probabilities across individuals.
- **Model Persistence**: If `save_path` is provided, fitted models are saved as `.pkl` files, enabling inspection or reuse.
- **Data Requirements**: Input must be a pandas DataFrame with columns for identifiers, periods, treatment, and covariates.

---

## Limitations

- **First Period Exclusion**: No weights are calculated for the first period, as there is no prior treatment to compare for switching.
- **Binary Treatment**: Assumes treatment is binary (0 or 1); non-binary treatments require modification.
- **Model Convergence**: Logistic regression may fail to converge with insufficient variation or collinearity, potentially yielding weights of 1.0.
- **Weight Stability**: Extreme weights may occur; NaN values are set to 1.0, but users should monitor for outliers.

---

## Relevance to Survival Analysis

The `StatsGlmLogit` class is often used in survival analysis to estimate treatment effects over time, as depicted in the provided image attachments (e.g., "Treatment Effect on Survival" graphs). The stabilized weights adjust for confounding due to treatment switching, enabling more accurate estimation of survival differences between treated and untreated groups. These weights can be applied in models like Cox proportional hazards to produce plots showing survival differences over follow-up periods, as seen in the described graphs.

In [5]:
class StatsGlmLogit:
    def __init__(self, save_path=None):
        self.save_path = save_path
        if save_path and not os.path.exists(save_path):
            os.makedirs(save_path)
    
    def fit(self, data, treatment_col, numerator_vars, denominator_vars, id_col, period_col):
        """Fit logistic regression models and calculate stabilized weights."""
        # Get periods where treatment can switch
        periods = sorted(data[period_col].unique())
        
        # Initialize DataFrame to store weights
        weights_df = pd.DataFrame()
        
        for period in periods[1:]:  # Skip the first period as there's no prior treatment to switch from
            # Get data for current period
            period_data = data[data[period_col] == period].copy()
            
            if len(period_data) == 0:
                continue
                
            # Get previous period for each individual
            prev_period = periods[periods.index(period) - 1]
            prev_data = data[data[period_col] == prev_period].copy()
            
            # Merge current with previous period data
            merged_data = pd.merge(
                period_data,
                prev_data[[id_col, treatment_col]],
                on=id_col,
                suffixes=('', '_prev')
            )
            
            # Identify individuals who switched treatment
            merged_data['switched'] = (merged_data[treatment_col] != merged_data[f"{treatment_col}_prev"]).astype(int)
            
            # Fit numerator model (simpler model)
            X_num = sm.add_constant(merged_data[numerator_vars])
            num_model = sm.Logit(merged_data['switched'], X_num).fit(disp=0)
            
            # Save model if requested
            if self.save_path:
                with open(os.path.join(self.save_path, f"num_model_period_{period}.pkl"), 'wb') as f:
                    pickle.dump(num_model, f)
            
            # Fit denominator model (full model)
            X_denom = sm.add_constant(merged_data[denominator_vars])
            denom_model = sm.Logit(merged_data['switched'], X_denom).fit(disp=0)
            
            # Save model if requested
            if self.save_path:
                with open(os.path.join(self.save_path, f"denom_model_period_{period}.pkl"), 'wb') as f:
                    pickle.dump(denom_model, f)
            
            # Calculate predicted probabilities
            num_probs = num_model.predict(X_num)
            denom_probs = denom_model.predict(X_denom)
            
            # Calculate stabilized weights
            merged_data['weight'] = num_probs / denom_probs
            merged_data['weight'] = merged_data['weight'].fillna(1.0)  # Handle division by zero
            
            # Add to weights DataFrame
            weights_df = pd.concat([weights_df, merged_data[[id_col, period_col, 'weight']]])
        
        return weights_df

In [6]:
def trial_sequence(estimand):
    """Create a new trial sequence object."""
    return TrialSequence(estimand=estimand)

In [7]:
def stats_glm_logit(save_path=None):
    """Create a logistic regression model fitter."""
    return StatsGlmLogit(save_path=save_path)

In [8]:
def save_to_datatable():
    """Handler for saving expanded data to a data table."""
    # This is a placeholder for the actual implementation
    def handler(data):
        return data
    return handler



In [9]:
def outcome_data(trial):
    """Extract outcome data from a trial sequence."""
    if trial.expansion is not None:
        return trial.expansion
    return pd.DataFrame()

In [10]:
def show_weight_models(trial):
    """Display information about weight models."""
    print("Weight Models Summary:")
    if hasattr(trial, 'switch_weights') and trial.switch_weights is not None:
        print(f"  Switch Weights: {len(trial.switch_weights)} rows")
    
    if hasattr(trial, 'censor_weights') and trial.censor_weights is not None:
        print(f"  Censor Weights: {len(trial.censor_weights)} rows")
    
    if hasattr(trial, 'combined_weights') and trial.combined_weights is not None:
        print(f"  Combined Weights: {len(trial.combined_weights)} rows")

In [11]:
if __name__ == "__main__":
    # Load data
    data_censored = pd.read_csv("data/data_censored.csv")
    
    # Create trial sequence objects
    trial_pp = trial_sequence(estimand="PP")  # Per-protocol
    trial_itt = trial_sequence(estimand="ITT")  # Intention-to-treat
    
    # Create directories
    trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
    os.makedirs(trial_pp_dir, exist_ok=True)
    
    trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")
    os.makedirs(trial_itt_dir, exist_ok=True)
    
    # Set data for both trial sequences
    trial_pp = trial_pp.set_data(
        data=data_censored,
        id="id",
        period="period",
        treatment="treatment",
        outcome="outcome",
        eligible="eligible"
    )
    
    trial_itt = trial_itt.set_data(
        data=data_censored,
        id="id",
        period="period",
        treatment="treatment",
        outcome="outcome",
        eligible="eligible"
    )
    
    # Set switch weight models
    trial_pp = trial_pp.set_switch_weight_model(
        numerator="~ age",
        denominator="~ age + x1 + x3",
        model_fitter=stats_glm_logit(save_path=os.path.join(trial_pp_dir, "switch_models"))
    )
    
    print(trial_pp)
    # Set censor weight models
    trial_pp = trial_pp.set_censor_weight_model(
        censor_event="censored",
        numerator="~ x2",
        denominator="~ x2 + x1",
        pool_models="none",
        model_fitter=stats_glm_logit(save_path=os.path.join(trial_pp_dir, "switch_models"))
    )
    
    trial_itt = trial_itt.set_censor_weight_model(
        censor_event="censored",
        numerator="~ x2",
        denominator="~ x2 + x1",
        pool_models="numerator",
        model_fitter=stats_glm_logit(save_path=os.path.join(trial_itt_dir, "switch_models"))
    )
    
    # Calculate weights
    trial_pp = trial_pp.calculate_weights()
    trial_itt = trial_itt.calculate_weights()
    
    # Show weight models
    show_weight_models(trial_itt)
    show_weight_models(trial_pp)
    
    # Set outcome models
    trial_pp = trial_pp.set_outcome_model()
    trial_itt = trial_itt.set_outcome_model(adjustment_terms="~ x2")
    
    # Set expansion options
    trial_pp = trial_pp.set_expansion_options(
        output=save_to_datatable(),
        chunk_size=500
    )
    
    trial_itt = trial_itt.set_expansion_options(
        output=save_to_datatable(),
        chunk_size=500
    )
    
    # Expand trials
    trial_pp = trial_pp.expand_trials()
    trial_itt = trial_itt.expand_trials()
    
    # Load expanded data and fit MSM
    trial_itt = trial_itt.load_expanded_data(seed=1234, p_control=0.5)
    trial_itt = trial_itt.fit_msm(
        weight_cols=["weight", "sample_weight"],
        modify_weights=lambda w: np.minimum(w, np.quantile(w, 0.99))  # Winsorization
    )
    
    # Prepare the prediction data
    prediction_data = outcome_data(trial_itt)
    prediction_data = prediction_data[prediction_data['trial_period'] == 1]

    # Add the 'const' column with value 1
    prediction_data['const'] = 1

    # Now call predict
    preds = trial_itt.predict(
        newdata=prediction_data,
        predict_times=list(range(11)),  # Assuming you want predictions from 0 to 10
        type="survival"
    )
    
    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(preds['difference']['followup_time'], preds['difference']['survival_diff'])
    plt.plot(preds['difference']['followup_time'], preds['difference']['2.5%'], 'r--')
    plt.plot(preds['difference']['followup_time'], preds['difference']['97.5%'], 'r--')
    plt.xlabel('Follow up')
    plt.ylabel('Survival difference')
    plt.title('Treatment Effect on Survival')
    plt.grid(True)
    plt.savefig('survival_difference.png')
    plt.close()

TrialSequence(estimand='PP', data=     id  period  treatment  x1        x2  x3        x4  age     age_s  \
0     1       0          1   1  1.146148   0  0.734203   36  0.083333   
1     1       1          1   1  0.002200   0  0.734203   37  0.166667   
2     1       2          1   0 -0.481762   0  0.734203   38  0.250000   
3     1       3          1   0  0.007872   0  0.734203   39  0.333333   
4     1       4          1   1  0.216054   0  0.734203   40  0.416667   
..   ..     ...        ...  ..       ...  ..       ...  ...       ...   
720  99       3          0   0 -0.747906   1  0.575268   68  2.750000   
721  99       4          0   0 -0.790056   1  0.575268   69  2.833333   
722  99       5          1   1  0.387429   1  0.575268   70  2.916667   
723  99       6          1   1 -0.033762   1  0.575268   71  3.000000   
724  99       7          0   0 -1.340497   1  0.575268   72  3.083333   

     outcome  censored  eligible  
0          0         0         1  
1          0       

C:\Users\Bryan Sanchez\AppData\Local\Temp\ipykernel_3748\3626396832.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_periods['switch_weight'].fillna(1.0, inplace=True)
C:\Users\Bryan Sanchez\AppData\Local\Temp\ipykernel_3748\3626396832.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

Weight Models Summary:
  Censor Weights: 725 rows
  Combined Weights: 1780 rows
Weight Models Summary:
  Switch Weights: 636 rows
  Censor Weights: 725 rows
  Combined Weights: 1780 rows


C:\Users\Bryan Sanchez\AppData\Local\Temp\ipykernel_3748\1380208114.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_data['const'] = 1
